In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from ray.tune.schedulers import PopulationBasedTraining
from sklearn.model_selection import KFold
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score

In [37]:
df=pd.read_csv('./cleaned_datasets/toy_data.csv',index_col=0)  

In [38]:
df

,x0,x1,x2,x3,x4,x5,y
0,0.366540,0.878304,1.038823,1.103452,156.014503,0,1626.664038
1,0.034019,1.776411,-1.344630,1.495489,160.756809,0,1614.286202
2,-0.078950,1.258357,-1.045738,0.695931,163.798515,0,1589.071117
3,-0.829983,-0.327983,1.312429,0.779775,174.602188,1,1626.477706
4,0.775547,3.168103,-0.701222,1.635968,148.929220,0,1625.306675
...,...,...,...,...,...,...,...
199995,1.141587,5.224048,0.743879,1.874382,161.278138,0,1703.145237
199996,0.949447,4.448228,0.005072,0.155069,158.457684,0,1594.895812
199997,0.664946,3.340872,-0.568699,1.328317,161.181416,0,1617.162981
199998,0.538717,2.695474,-0.621813,1.137954,160.854668,0,1610.381437


In [39]:
y = df.pop('y')
X = df
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.9)
#subsampling beacuse so much data
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train,test_size=0.2)




In [40]:
X_train

,x0,x1,x2,x3,x4,x5
108437,0.794318,3.137060,-0.475899,0.765490,162.943422,0
12498,2.279754,6.048136,-0.035587,0.312561,161.306167,0
9035,1.872995,6.548175,0.452450,1.510092,167.111321,1
124697,-0.407031,0.112801,-0.022373,-0.067884,158.762275,0
98312,0.721974,2.898678,-0.928320,1.182668,154.310961,1
...,...,...,...,...,...,...
136899,0.054737,0.909058,0.446604,0.575540,160.572766,0
56480,0.475884,2.109485,-0.894317,1.164541,160.051555,0
22542,0.647619,2.100463,1.155955,1.624781,162.286484,0
136596,1.200933,3.666515,-0.575042,1.555233,169.364080,1


In [41]:
y_train

108437    1596.168146
12498     1605.049068
9035      1714.447955
124697    1573.048307
98312     1633.086808
             ...     
136899    1604.322887
56480     1608.070479
22542     1666.521438
136596    1652.800862
154103    1822.909537
Name: y, Length: 16000, dtype: float64

In [42]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [43]:
class DataFrameDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = torch.tensor(X_data, dtype=torch.float32)
        self.y_data = torch.tensor(y_data, dtype=torch.float32)

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [44]:
def create_dataloader(X_train, X_test, y_train, y_test, batch_size=32):
    # Create custom Dataset objects for train and test data
    train_dataset = DataFrameDataset(X_train, y_train)
    test_dataset = DataFrameDataset(X_test, y_test)

    # Create DataLoaders for train and test data
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [45]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, hidden_layers, hidden_units, dropout, dropout_array, input_dim, output_dim, activation=nn.ReLU(),norm=False):
        if hidden_layers != len(hidden_units):
            print("Error: wrong size of hidden_layers or hidden_units")
            return
        layers = []
        i = 0
        if norm:
            layers.append(nn.BatchNorm1d(input_dim))

        super(Net, self).__init__()
        
        layers.append(nn.Linear(input_dim, hidden_units[i]))
        layers.append(activation)
        if dropout:
            layers.append(nn.Dropout(dropout_array[i]))

        for _ in range(hidden_layers - 1):
            i += 1
            layers.append(nn.Linear(hidden_units[i-1], hidden_units[i]))
            layers.append(activation)
            if dropout:
                layers.append(nn.Dropout(dropout_array[i]))

        layers.append(nn.Linear(hidden_units[-1], output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.net(x)


In [46]:
model = Net(hidden_layers=3, hidden_units=[24, 24, 24], dropout=True, dropout_array=[0.2, 0.3, 0.4], input_dim=10, output_dim=1, activation=nn.Sigmoid(),norm=True)


In [47]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = model.to(device)

In [48]:
summary(model,input_size=(10,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                   [-1, 10]              20
            Linear-2                   [-1, 24]             264
           Sigmoid-3                   [-1, 24]               0
           Dropout-4                   [-1, 24]               0
            Linear-5                   [-1, 24]             600
           Sigmoid-6                   [-1, 24]               0
           Dropout-7                   [-1, 24]               0
            Linear-8                   [-1, 24]             600
           Sigmoid-9                   [-1, 24]               0
          Dropout-10                   [-1, 24]               0
           Linear-11                    [-1, 1]              25
Total params: 1,509
Trainable params: 1,509
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/back

In [49]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [50]:
def train(config, checkpoint_dir=None):
    hidden_layers=config["hidden_layers"]
    hidden_units=config["hidden_units"]
    dropout=config["dropout"]
    dropout_array=config["dropout_array"]
    activation=config["activation"]
    norm=config["norm"]
    lr=config["lr"]
    input_dim=X_train.shape[1]
    output_dim=1
    
    train_ds = TensorDataset(torch.Tensor(X_train_scaled), torch.Tensor(y_train.to_numpy()))
    test_ds = TensorDataset(torch.Tensor(X_test_scaled), torch.Tensor(y_test.to_numpy()))
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    fold = 0
    avg_loss = 0
    avg_r2 = 0

    for train_idx, val_idx in kfold.split(train_ds):
        print("This is fold :" ,fold)
        #trainloader, testloader = create_dataloader(X_train_scaled, X_test_scaled, y_train.values, y_test.values, batch_size=config["batch_size"])
        net =Net(hidden_layers=hidden_layers, hidden_units=hidden_units, dropout=dropout, dropout_array=dropout_array, input_dim=input_dim, output_dim=output_dim, activation=activation,norm=norm)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        net.to(device)
        criterion = RMSELoss()
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])
        train_subset = torch.utils.data.Subset(train_ds, train_idx)
        val_subset = torch.utils.data.Subset(train_ds, val_idx)

        train_loader = DataLoader(train_subset, batch_size=config["batch_size"], shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=config["batch_size"])
    
        for epoch in range(config["epochs"]):
            running_loss = 0.0
            train_loss=0.0
            val_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            train_loss=running_loss/len(train_loader)
               
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()      
            avg_loss += val_loss / len(val_loader)
            y_true = y_test.to_numpy()
            y_pred = net(torch.Tensor(X_test_scaled).to(device)).cpu()
            r2 = r2_score(y_true, y_pred)
            print(r2)
            tune.report(r2_curr=r2)
            avg_r2 += r2
        fold += 1
        

    avg_loss /= fold
    avg_r2 /= fold
    print("Average loss across folds: ", avg_loss)
    print("Average R2 score across folds: ", avg_r2)
    tune.report(loss=avg_loss,train_loss=train_loss,r2_score=avg_r2,r2_curr=r2)

In [51]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [52]:
device

'cuda'

In [53]:
def generate_hidden_units(hidden_layers):
    return np.random.choice(a=[16,32,64,128], size=hidden_layers).tolist()

search_space = {
    "batch_size": tune.choice([128,256,512, 1024, 2048]),
    #"batch_size": tune.choice([1024, 2048]),
    "lr": tune.loguniform(1e-7, 1e-2),
    "epochs": tune.choice([10,25,50,75]),
    #"epochs": tune.choice([5]),
    "hidden_layers": tune.randint(1, 4),
    "hidden_units": tune.sample_from(lambda spec: generate_hidden_units(spec.config.hidden_layers)),
    "dropout": tune.choice([True, False]),
    "dropout_array": tune.sample_from(lambda spec: np.random.uniform(low=0.1, high=0.8, size=spec.config.hidden_layers).tolist()),
    "activation": tune.choice([nn.ReLU()]),
    "norm": tune.choice([True, False])
}

In [54]:
scheduler = ASHAScheduler(metric="r2_curr", mode="max", grace_period=2, reduction_factor=2)
"""
pbt_scheduler = PopulationBasedTraining(
    time_attr='training_iteration',
    metric='loss',
    mode='min',
    perturbation_interval=1,
    hyperparam_mutations=search_space
)
"""
reporter = CLIReporter(metric_columns=["loss", "training_iteration","train_loss"])

In [55]:
analysis = tune.run(
        train,
        resources_per_trial={"cpu": 8, "gpu": 1 if torch.cuda.is_available() else 0},
        config=search_space,
        num_samples=100,
        scheduler=scheduler,
        progress_reporter=reporter)

print("Best hyperparameters found were: ", analysis.get_best_config(metric="r2_score", mode="max"))


== Status ==
Current time: 2023-05-11 13:48:04 (running for 00:00:00.73)
Memory usage on this node: 11.1/15.9 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.15 GiB heap, 0.0/1.57 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-05-11_13-48-03
Number of trials: 17/100 (16 PENDING, 1 RUNNING)
+-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+
| Trial name        | status   | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |
|-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+------

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,r2_curr,r2_score,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,training_iteration,trial_id,warmup_time
train_afcdf_00000,2023-05-11_13-48-38,True,,5eb4a1425da04a2abe5c0278b4dbb39e,"0_activation=ReLU,batch_size=1024,dropout=True,dropout_array=0_6003013014482692,epochs=50,hidden_layers=1,hidden_units=128,lr=0.0000,norm=True",DESKTOP-8SILQJU,6,1653.3859069824218,127.0.0.1,11956,-1043.84,-1043.7719990699065,31.2756,0.00500488,31.2756,1683805718,0,,1653.506845327524,6,afcdf_00000,0.00400352
train_afcdf_00001,2023-05-11_13-49-15,True,,14bef0838bdb46f7a359d86f24fd536f,"1_activation=ReLU,batch_size=1024,dropout=True,dropout_array=0_5401695042904022_0_733468146250674_0_10368916663620215,epochs=50,hidden_layers=3,hidden_units=16_32_64,lr=0.0000,norm=False",DESKTOP-8SILQJU,6,1653.298583984375,127.0.0.1,1976,-1043.81,-1043.6700705913486,32.7155,0.00500464,32.7155,1683805755,0,,1653.6327655498799,6,afcdf_00001,0.00500464
train_afcdf_00002,2023-05-11_13-49-33,True,,1d4e08df0f4843b690cf166d7feb037b,"2_activation=ReLU,batch_size=128,dropout=True,dropout_array=0_273763772321294_0_7841580279859545,epochs=10,hidden_layers=2,hidden_units=16_128,lr=0.0000,norm=True",DESKTOP-8SILQJU,6,1649.9815576171873,127.0.0.1,10596,-1039.87,-1039.167633196257,14.247,0.00300312,14.247,1683805773,0,,1650.91830078125,6,afcdf_00002,0.00400472
train_afcdf_00003,2023-05-11_13-49-50,True,,df5cca2dbb764ad2ba797bc4100fc8a9,,DESKTOP-8SILQJU,4,,127.0.0.1,22912,-1043.69,,13.5711,3.13385,13.5711,1683805790,0,,,4,afcdf_00003,0.00400496
train_afcdf_00004,2023-05-11_13-50-00,True,,5e51e5c4576644d8aed63bc90120cf6b,,DESKTOP-8SILQJU,4,,127.0.0.1,19020,-1043.56,,6.39291,1.23889,6.39291,1683805800,0,,,4,afcdf_00004,0.00400472
train_afcdf_00005,2023-05-11_13-50-34,True,,7b0135b3f5ff4782a5be2dad54999602,"5_activation=ReLU,batch_size=2048,dropout=True,dropout_array=0_5023359389986466,epochs=50,hidden_layers=1,hidden_units=64,lr=0.0010,norm=False",DESKTOP-8SILQJU,6,1626.3257202148438,127.0.0.1,23284,-1008.39,-1009.4829107599396,29.8477,0.00500441,29.8477,1683805834,0,,1626.288120814732,6,afcdf_00005,0.00400305
train_afcdf_00006,2023-05-11_13-50-44,True,,54748da225a44570b4a6f65577ccd3e6,,DESKTOP-8SILQJU,2,,127.0.0.1,3576,-1044.13,,7.16052,2.73249,7.16052,1683805844,0,,,2,afcdf_00006,0.00400257
train_afcdf_00007,2023-05-11_13-51-52,True,,89204cc7281a40309f16c0cc55dd3243,"7_activation=ReLU,batch_size=256,dropout=True,dropout_array=0_69414351202374_0_6773658370965412,epochs=75,hidden_layers=2,hidden_units=64_32,lr=0.0000,norm=False",DESKTOP-8SILQJU,6,1653.3834754356972,127.0.0.1,23372,-1043.53,-1043.5054416971573,64.0287,0.00600553,64.0287,1683805912,0,,1653.4694311523438,6,afcdf_00007,0.00400472
train_afcdf_00008,2023-05-11_13-52-43,True,,581fe653f1754caf93dbc816f0e7579f,"8_activation=ReLU,batch_size=2048,dropout=False,dropout_array=0_1668111285946674_0_6629670096249307_0_41366338810105774,epochs=75,hidden_layers=3,hidden_units=128_32_16,lr=0.0000,norm=True",DESKTOP-8SILQJU,6,1653.45361328125,127.0.0.1,10780,-1043.83,-1043.5033986531985,47.261,0.00700665,47.261,1683805963,0,,1654.238804408482,6,afcdf_00008,0.00400448
train_afcdf_00009,2023-05-11_13-53-23,True,,0d11f04910fc40aa8cb40509167e6804,,DESKTOP-8SILQJU,4,,127.0.0.1,8488,-1043.44,,35.8425,8.69975,35.8425,1683806003,0,,,4,afcdf_00009,0.00400329


(train pid=11956) -1043.5677443272175
(train pid=11956) This is fold : 1
== Status ==
Current time: 2023-05-11 13:48:19 (running for 00:00:16.50)
Memory usage on this node: 13.3/15.9 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.15 GiB heap, 0.0/1.57 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-05-11_13-48-03
Number of trials: 18/100 (17 PENDING, 1 RUNNING)
+-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+----------------------+
| Trial name        | status   | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |   training_iteration |
|-------------------+----------

(train pid=11956) -1044.0027343838733
(train pid=11956) This is fold : 3
== Status ==
Current time: 2023-05-11 13:48:31 (running for 00:00:28.31)
Memory usage on this node: 13.5/15.9 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: -1043.7805353554365
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.15 GiB heap, 0.0/1.57 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-05-11_13-48-03
Number of trials: 18/100 (17 PENDING, 1 RUNNING)
+-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+----------------------+
| Trial name        | status   | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |   training_iteration |
|---------------

2023-05-11 13:48:38,773	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': ReLU()}


...

(train pid=10232) -59.0049434909484
(train pid=10232) This is fold : 1
== Status ==
Current time: 2023-05-11 14:40:34 (running for 00:52:31.57)
Memory usage on this node: 10.9/15.9 GiB 
Using AsyncHyperBand: num_stopped=57
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: -149.74802259809334 | Iter 2.000: -1041.6751583416954
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.15 GiB heap, 0.0/1.57 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-05-11_13-48-03
Number of trials: 100/100 (1 RUNNING, 99 TERMINATED)
+-------------------+------------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+------------+----------------------+--------------+
| Trial name        | status     | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr 

(train pid=10232) -54.99593388020503
(train pid=10232) This is fold : 3
== Status ==
Current time: 2023-05-11 14:40:50 (running for 00:52:47.32)
Memory usage on this node: 10.9/15.9 GiB 
Using AsyncHyperBand: num_stopped=57
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: -149.74802259809334 | Iter 2.000: -1041.4641306046983
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.15 GiB heap, 0.0/1.57 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-05-11_13-48-03
Number of trials: 100/100 (1 RUNNING, 99 TERMINATED)
+-------------------+------------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+------------+----------------------+--------------+
| Trial name        | status     | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr

2023-05-11 14:41:01,704	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': ReLU()}
2023-05-11 14:41:01,864	INFO tune.py:762 -- Total run time: 3178.50 seconds (3178.30 seconds for the tuning loop).


(train pid=10232) -57.86939805031105
(train pid=10232) Average loss across folds:  380.1743992396763
(train pid=10232) Average R2 score across folds:  -54.802138397320775
== Status ==
Current time: 2023-05-11 14:41:01 (running for 00:52:58.33)
Memory usage on this node: 11.0/15.9 GiB 
Using AsyncHyperBand: num_stopped=57
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: -143.31398991295933 | Iter 2.000: -1041.4641306046983
Resources requested: 0/16 CPUs, 0/1 GPUs, 0.0/3.15 GiB heap, 0.0/1.57 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-05-11_13-48-03
Number of trials: 100/100 (100 TERMINATED)
+-------------------+------------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+-----------------+-------------+--------+------------+----------------------+--------------+
| Trial name        | status     | loc             | activation   |   batch_size | dropou

Best hyperparameters found were:  {'batch_size': 128, 'lr': 0.0011035844390021956, 'epochs': 25, 'hidden_layers': 3, 'hidden_units': [128, 64, 32], 'dropout': False, 'dropout_array': [0.4488934740309861, 0.7072925570890253, 0.6445757596805983], 'activation': ReLU(), 'norm': False}


In [56]:
analysis.results_df.sort_values(by=['r2_score', 'loss'],na_position='first').tail(10)

,loss,train_loss,r2_score,r2_curr,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,...,experiment_tag,config/batch_size,config/lr,config/epochs,config/hidden_layers,config/hidden_units,config/dropout,config/dropout_array,config/activation,config/norm
trial_id,,,,,,,,,,,,,,,,,,,,,
afcdf_00076,18.390404,18.304601,0.883137,0.884567,0.020021,True,None,None,6,c85fc8232b314dc7a589c453a9c55f23,...,"76_activation=ReLU,batch_size=128,dropout=Fals...",128,0.001008,50,1,[16],False,[0.525283085391477],ReLU(),True
afcdf_00029,17.255343,32.084290,0.894407,0.890765,0.011010,True,None,None,6,65a63e70c0284c09a4bf58c82a9fa3f6,...,"29_activation=ReLU,batch_size=2048,dropout=Fal...",2048,0.006635,75,3,"[16, 64, 64]",False,"[0.7566266150942558, 0.7688403952064711, 0.207...",ReLU(),True
afcdf_00042,20.913761,14.228293,0.905543,0.943382,0.013013,True,None,None,6,05ffde14e076429b95a0f3823e4dad06,...,"42_activation=ReLU,batch_size=128,dropout=Fals...",128,0.001964,25,3,"[16, 16, 16]",False,"[0.19977537902913794, 0.10031443539851174, 0.7...",ReLU(),True
afcdf_00036,10.539562,8.399062,0.957453,0.970053,0.012962,True,None,None,6,208cb1d8cd694ce18b41d9b503f1a9fb,...,"36_activation=ReLU,batch_size=2048,dropout=Fal...",2048,0.006773,50,2,"[32, 128]",False,"[0.3697678846139516, 0.3005367436852363]",ReLU(),False
afcdf_00072,9.097262,13.893605,0.964382,0.930188,0.019017,True,None,None,6,dc2230b785394b42bef071f676c28a25,...,"72_activation=ReLU,batch_size=1024,dropout=Fal...",1024,0.003017,25,3,"[128, 64, 32]",False,"[0.15408251601856204, 0.7875896166426757, 0.48...",ReLU(),False
afcdf_00057,5.933727,7.020556,0.984198,0.990288,0.016014,True,None,None,6,d7db97c29588409f974e03d227978587,...,"57_activation=ReLU,batch_size=128,dropout=Fals...",128,0.007115,50,3,"[64, 32, 64]",False,"[0.7347949215776113, 0.2609548109270827, 0.312...",ReLU(),False
afcdf_00049,5.638276,4.911206,0.986957,0.994646,0.014520,True,None,None,6,1cb4cba362334243b1995cd587fffc62,...,"49_activation=ReLU,batch_size=1024,dropout=Fal...",1024,0.005061,50,3,"[64, 64, 32]",False,"[0.3317530913669876, 0.11629840389042367, 0.72...",ReLU(),False
afcdf_00013,5.156273,6.287821,0.989338,0.985348,0.008008,True,None,None,6,908c5ddc44804a3cb927dac0544d7c8d,...,"13_activation=ReLU,batch_size=256,dropout=Fals...",256,0.003527,50,3,"[32, 128, 64]",False,"[0.5472732889485082, 0.24840777233725236, 0.74...",ReLU(),False
afcdf_00089,5.189347,5.199367,0.989884,0.990285,0.022017,True,None,None,6,f314140a594a4c3b89adc77db668fbd0,...,"89_activation=ReLU,batch_size=256,dropout=Fals...",256,0.001409,75,3,"[16, 16, 32]",False,"[0.2049334961388014, 0.24601140077131523, 0.72...",ReLU(),False


In [24]:
x=np.linspace(400,10000,101) 
plt.scatter(y_test_pred.detach().numpy(), y_test_con)
plt.plot(x,x,'r')
plt.show()

NameError: name 'y_test_pred' is not defined

In [57]:
cfg = {'batch_size': 128, 'lr': 0.001104, 'epochs': 25, 'hidden_layers': 3, 'hidden_units': [128, 64, 32], 'dropout': False, 'dropout_array': [0.38283509966903684, 0.14655095633529086, 0.5393755103727268], 'activation': nn.ReLU(), 'norm': False}
hidden_layers=cfg["hidden_layers"]
hidden_units=cfg["hidden_units"]
dropout=cfg["dropout"]
dropout_array=cfg["dropout_array"]
activation=cfg["activation"]
norm=cfg["norm"]
lr=cfg["lr"]

In [66]:
net =Net(hidden_layers=hidden_layers, hidden_units=hidden_units, dropout=dropout, dropout_array=dropout_array, input_dim=6, output_dim=1, activation=activation,norm=norm)

In [67]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
net = net.to(device)

In [68]:
summary(net,input_size=(6,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]             896
              ReLU-2                  [-1, 128]               0
            Linear-3                   [-1, 64]           8,256
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 32]           2,080
              ReLU-6                   [-1, 32]               0
            Linear-7                    [-1, 1]              33
Total params: 11,265
Trainable params: 11,265
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.04
Estimated Total Size (MB): 0.05
----------------------------------------------------------------


In [69]:
best_config={'batch_size': 128, 'lr': 0.001104, 'epochs': 25, 'hidden_layers': 3, 'hidden_units': [128, 64, 32], 'dropout': False, 'dropout_array': [0.38283509966903684, 0.14655095633529086, 0.5393755103727268], 'activation': nn.ReLU(), 'norm': False}

In [70]:
best_config

{'batch_size': 128,
 'lr': 0.001104,
 'epochs': 25,
 'hidden_layers': 3,
 'hidden_units': [128, 64, 32],
 'dropout': False,
 'dropout_array': [0.38283509966903684,
  0.14655095633529086,
  0.5393755103727268],
 'activation': ReLU(),
 'norm': False}

In [71]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def cross_val_train(config, n_splits=5):
    kf = KFold(n_splits=n_splits)
    r2_scores = []
    rmse_scores = []

    for train_index, test_index in kf.split(X_train_scaled, y_train.values):
        X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_test_cv = y_train.values[train_index], y_train.values[test_index]

        train_loader, test_loader = create_dataloader(X_train_cv, X_test_cv, y_train_cv, y_test_cv, batch_size=config["batch_size"])

        input_dim = X_train_cv.shape[1]
        output_dim = 1
        net = Net(hidden_layers=config["hidden_layers"], hidden_units=config["hidden_units"], dropout=config["dropout"], dropout_array=config["dropout_array"], input_dim=input_dim, output_dim=output_dim, activation=config["activation"], norm=config["norm"])

        device = "cuda" if torch.cuda.is_available() else "cpu"
        net.to(device)

        criterion = RMSELoss()
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])

        for epoch in range(config["epochs"]):
            running_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            

        with torch.no_grad():
            test_loss = 0.0
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

            y_pred = net(torch.Tensor(X_test_cv).to(device)).cpu().numpy()
            r2_scores.append(r2_score(y_test_cv, y_pred))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test_cv, y_pred)))
            #print(r2_scores)
            #print(rmse_scores)

    avg_r2 = np.mean(r2_scores)
    avg_rmse = np.mean(rmse_scores)

    return avg_r2, avg_rmse
avg_r2, avg_rmse = cross_val_train(best_config)
print(f"Average R2 score: {avg_r2}")
print(f"Average RMSE: {avg_rmse}")


Average R2 score: 0.9947039918071294
Average RMSE: 3.733145570373046


In [ ]:
!pip install deap


In [72]:
best_config

{'batch_size': 128,
 'lr': 0.001104,
 'epochs': 25,
 'hidden_layers': 3,
 'hidden_units': [128, 64, 32],
 'dropout': False,
 'dropout_array': [0.38283509966903684,
  0.14655095633529086,
  0.5393755103727268],
 'activation': ReLU(),
 'norm': False}

In [73]:
import numpy as np
from skopt import gp_minimize

fixed_config = {
    'hidden_layers': 3,
 'hidden_units': [128, 64, 32],
 'dropout': False,
 'dropout_array': [0.38283509966903684,
  0.14655095633529086,
  0.5393755103727268],
 'activation': nn.ReLU(),
 'norm': False
}

# The objective function to minimize (negative R^2)
def objective(params):
    batch_size, lr, epochs = params
    config = {'batch_size': int(batch_size), 'lr': lr, 'epochs': int(epochs)}
    config.update(fixed_config)

    r2, rmse = cross_val_train(config)
    return rmse  # Minimize negative R^2 to maximize R^2

# Hyperparameter search space
space = [
    (25, 100),  # batch_size
    (1e-4, 1e-2, 'log-uniform'),  # lr
    (25, 100)  # epochs
]

# Bayesian optimization using Gaussian Process
result = gp_minimize(
    func=objective,
    dimensions=space,
    n_calls=50,  # Number of iterations
    verbose=True
)

# Print the optimized hyperparameters
print("Best hyperparameters found:")
print("Batch size:", result.x[0])
print("Learning rate:", result.x[1])
print("Epochs:", result.x[2])


Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 137.6874
Function value obtained: 3.5511
Current minimum: 3.5511
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 90.9316
Function value obtained: 3.1233
Current minimum: 3.1233
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 81.5448
Function value obtained: 3.0918
Current minimum: 3.0918
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 119.6404
Function value obtained: 4.3798
Current minimum: 3.0918
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 100.6789
Function value obtained: 10.4436
Current minimum: 3.0918
Iteration No: 6 started. E

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 317.0555
Function value obtained: 2.2997
Current minimum: 2.2040
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 112.9303
Function value obtained: 2.8374
Current minimum: 2.2040
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 365.2599
Function value obtained: 2.2329
Current minimum: 2.2040
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 149.3186
Function value obtained: 2.5103
Current minimum: 2.2040
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 294.0778
Function value obtained: 2.2862
Current minimum: 2.2040
Iteration No: 46 st

In [74]:
best_config_final={'batch_size': 25, 'lr': 0.0004990515469382572, 'epochs': 100, 'hidden_layers': 3, 'hidden_units': [128, 64, 32], 'dropout': False, 'dropout_array': [0.38283509966903684, 0.14655095633529086, 0.5393755103727268], 'activation': nn.ReLU(), 'norm': False}